create a samll and big versions of toy example data

# 1. Simpson's Paradox
- Create toy data where the paradox is evident
- Fit a naïve regression where the relation is not intuitive
- Then show what happens when a confounder is added (direction is changed)
- Throwing all the variables in kind of works but specifying a causal DAG will get the estimates correctly

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, when, lit, col
import pyspark.sql.functions as F

StatementMeta(ac476836-85ff-4d1a-95f2-f23a57b994fa, 0, 6, Finished, Available, Finished)

In [5]:
# Initialize Spark session

spark = SparkSession.builder.appName("ToyExamplePySpark").getOrCreate()
# Set random seed
#spark.sparkContext.setRandomSeed(853210)

StatementMeta(ac476836-85ff-4d1a-95f2-f23a57b994fa, 0, 10, Finished, Available, Finished)

In [6]:
# Create a DataFrame with the desired number of samples
def genearate_toy_data(n_samples = 2000, out_fn):
df = spark.range(n_samples)

# Generate toy data
df = df.withColumn("Winter_Ind", F.when(F.rand() < 0.24, 1).otherwise(0))

df = df.withColumn("Rain_Ind", 
                   F.when(F.rand() < (0.2 + col("Winter_Ind") * 0.3), 1).otherwise(0))

df = df.withColumn("Speed_KMpH", 
                   F.randn() * 0.7 + (60 - col("Rain_Ind") * 0.9))

df = df.withColumn("Fuel_Consumption_LpKM", 
                   F.randn() * 0.5 + (50 + col("Speed_KMpH") / 4 + col("Rain_Ind") * 2.1))

# Select only the columns we need
toy_example = df.select("Winter_Ind", "Rain_Ind", "Speed_KMpH", "Fuel_Consumption_LpKM")

# Show the first few rows
toy_example.show(5)

# Save toy_example as Parquet file locally
toy_example.write.parquet("toy_example.parquet")

# To use the saved Parquet file later, you can read it like this:
# loaded_toy_example = spark.read.parquet("toy_example.parquet")



StatementMeta(ac476836-85ff-4d1a-95f2-f23a57b994fa, 0, 11, Finished, Available, Finished)

+----------+--------+------------------+---------------------+
|Winter_Ind|Rain_Ind|        Speed_KMpH|Fuel_Consumption_LpKM|
+----------+--------+------------------+---------------------+
|         0|       0|59.876793912505036|    65.63448970803802|
|         0|       0|60.386061523112275|    64.84484197561234|
|         1|       1| 58.19972901532991|    66.01088537561259|
|         0|       0|60.199543106452296|    65.27469509708527|
|         1|       1| 59.65328730349418|    67.95564542203937|
+----------+--------+------------------+---------------------+
only showing top 5 rows



Exception in thread Thread-9 (_upload_with_retries):
Traceback (most recent call last):
  File "/home/trusted-service-user/cluster-env/env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/trusted-service-user/cluster-env/env/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages/synapse_context_manager.py", line 240, in _upload_with_retries
    do_one_upload_iteration()
  File "/home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages/synapse_context_manager.py", line 217, in do_one_upload_iteration
    with open(local_path, 'rb') as local_log, open(mnt_path, 'ab') as remote_log:
FileNotFoundError: [Errno 2] No such file or directory: '/synfs/notebook/AmlJobLogs/dcid.0fc99243-5540-4fd5-947c-3e035e1cef3c/logs/azureml/driver/stdout'
Exception in thread Thread-8 (_upload_with_retries):
Traceback (most recent call last)

In [ ]:
# Stop the Spark session
spark.stop()